# Getting your hands dirty with LDA (Using the Foursquare API)

First things first, we want to develop a classifier that can detect if a user comment is misapplied to the venue. Think of all the times you've gone to an Amazon item and saw reviews that looked like the image below, "Used but in very good condition", "fast shipping" are these relevant to a product review? Or are they more appropriate for seller reviews? 

We're going to see if we can develop a comment classifier to detect which kinds of comments may be more appropriate for which venue type, and thus provide some indication on the "appropriateness" of the comment that can be used for further review, thus (we hope), improving the overall quality of Foursquare as a reviews platform

As an example: "This place got me drunk" would be a comment you'd expect to see for a store category "bar" 
Yet, "I want to have my wedding here" is definitely not a comment you'd expect to see at a "bar" category, more likely you'd see such a category associated with a park, or other such venue. 

In [2]:
from IPython.display import Image
Image('F:/test/Bad Amazon Reviews.PNG')

Use the previously built table-creator on a few categories of Foursquare venues. 

Find the category list code in the following page: https://developer.foursquare.com/categorytree

In [6]:
import foursquare
import json
import pandas as pd
import unicodedata

CLIENT_ID = 'VSLROL5XMTRYZK3RJSESVVMSMAV3WN1LJF1BEW2RIJCDQOTY'
CLIENT_SECRET = 'GKNMUYT1MLLJUBUZDG4KQ14YOMNZZBBIUAZBVGSBYYPAMKDX'
client = foursquare.Foursquare(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)

ImportError: No module named foursquare

Load all of your venues into a dataframe. Make sure that you have at least 4-6 categories of venues included in your data so we can seperate them out later with LDA.

In [7]:
start_cats = ['4bf58dd8d48988d130941735', '4d4b7105d754a06374d81259', '4d4b7105d754a06375d81259', '4d4b7104d754a06370d81259']
venue_ids = []
categories = []

for cat in start_cats:
    temp = client.venues.search(params={'limit':50, 'near': 'Seattle, WA', 'radius':'5000', 'categoryId': cat})
    x = json.dumps(temp, indent = 4)
    dicty = json.loads(x)
    for venue in dicty['venues']:
        venue_ids.append(venue['id'])
        categories.append(cat)

NameError: name 'client' is not defined

# Constructing features using string parsing and elementary text mining

Create a normalizer function that will take the comments, lowercase all uppercase letters, split each comment into an array/list/vector of words, and remove stop words. 

In [1]:
import re
import os
import codecs
from sklearn import feature_extraction
# import mpld3            
import numpy as np
from textblob import TextBlob


comments = []
count = 0
final_cats = []

for ID in venue_ids:
    venue_details = client.venues(VENUE_ID = ID)
    path_to_text = venue_details['venue']['tips']['groups'][0]['items']
    temp = []
    for comment in path_to_text:
        final_cats.append(categories[count])
        comments.append(comment['text'])
    count += 1

print len(final_cats)
print len(comments)

Write a normalizing method, one that will convert all letters in a string into lower case, and remove stopwords. 

In [ ]:
# Count Vectorizer Does This

Use the normalizer that you created to build a 'stripped' down comments array/vector/list (however you did it)

In [ ]:
# CountVectorizer does this.

Vectorize your feature set into a dense matrix representation using Countvectorizer from sklearn

A helpful reference: http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectKBest

test = df[df['category'] != 0]

v = CountVectorizer(stop_words='english', 
                      token_pattern='[a-zA-Z]+', 
                      max_df=0.5, min_df=2)
X = pd.DataFrame(v.fit_transform(test['comments']).todense())
X.columns = v.get_feature_names()
y = test['category']

# Lots of data so lets select some of the more important stuff
kbest = SelectKBest(k=400)
kbest.fit_transform(X, y)

# boolean mask to get column names
mask = kbest.get_support()
voc = pd.DataFrame(np.asarray(v.get_feature_names())[mask])

# Vectorize with the vocab from above
v2 = CountVectorizer(vocabulary=voc[0])
X = pd.DataFrame(v2.fit_transform(test['comments']).todense())
X.columns = v2.get_feature_names()

# Performing the LDA

Construct your LDA with the pieces above by fitting the model to your feature set and target

In [5]:
import scipy

In [6]:

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split


lda = LDA(n_components=4)
# This gives us new features, components.
components = lda.fit_transform(X,y)
print lda.score(X, y)
y_pred = lda.predict(X)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
X_train, X_test, y_train, y_test = train_test_split(X.values, y, stratify=y, test_size=0.30, random_state=42)

lda = LDA(n_components=4)
components_2 = lda.fit_transform(X_train,y_train)
print lda.score(X_test, y_test)
y_pred = lda.predict(X_test)
print classification_report(y_test, y_pred)
confusion_matrix(y_test, y_pred)

# Visualizing Results

Now print the LDA and at least a few of the categories on a 2-D discriminant chart. Do you think the method is seperating the categories very well? Should we tune it more? Would it be a good indicator of "inappropriate commenting"? 

In [7]:
%matplotlib inline
import matplotlib.pyplot as plt
import random

X_new = lda.transform(X)

In [ ]:
import seaborn as sns
sns.pairplot(plot_df, hue='y')